## Dataset Pre-processing
### Download Beauty Dataset from [Amazon 2018](https://nijianmo.github.io/amazon/index.html)

In [1]:
!python data/data_process.py --file_path data/Beauty/All_Beauty_5.json.gz --output_path data/Beauty

5269it [00:00, 109003.41it/s]
100%|█████████████████████████████████████| 991/991 [00:00<00:00, 443602.48it/s]


## Load Dependencies and Hyper-Parameters

In [2]:
import torch

from src.models import model_factory
from src.dataloaders import dataloader_factory
from src.datasets import dataset_factory
from src.trainers import trainer_factory
from src.utils.utils import *
from src.utils.options import parser

In [3]:
args = parser.parse_args([])

args.data_path = 'data/Beauty'
args.num_epochs = 100
args.trm_max_len = 50

ckpt_root = setup_train(args)

Folder created: /home/zhankui/FastRec/experiments/test_2021-04-01_11
{'adam_epsilon': 1e-06,
 'best_metric': 'NDCG@10',
 'data_path': 'data/Beauty',
 'dataloader_code': 'sasrec',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'item',
 'dataset_split_seed': 98765,
 'device': 'cpu',
 'device_idx': '0',
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'global_epochs': 1000,
 'local_epochs': 10,
 'lr': 0.001,
 'metric_ks': [5,
               10,
               20],
 'model_code': 'sasrec',
 'model_init_seed': 0,
 'num_epochs': 100,
 'num_gpu': 1,
 'optimizer': 'Adam',
 'split': 'leave_one_out',
 'subset_size': 1000,
 'test_batch_size': 64,
 'test_negative_sample_size': 100,
 'test_negative_sampler_code': 'random',
 'test_negative_sampling_seed': 98765,
 'train_batch_size': 64,
 'train_negative_sample_size': 100,
 'train_negative_sampler_code': 'random',
 'train_negative_sampling_seed': 0,
 'trainer_code': 'sasrec_sample',
 'trm_att_dropout': 0.2,
 'trm_dropout': 0.2,

## Build Dataset

In [4]:
dataset = dataset_factory(args)
train_loader, val_loader, test_loader, dataset = dataloader_factory(args, dataset)

## Model Setup
We load [SASRec Model](https://arxiv.org/abs/1808.09781) for sequential recommendation

In [5]:
model = model_factory(args)
if args.load_pretrained_weights is not None:
    print("weights loading from %s ..." % args.load_pretrained_weights)
    model = load_pretrained_weights(model, args.load_pretrained_weights)
print("Model size:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print(model)

Model size: 37950
SASRecModel(
  (loss): BCEWithLogitsLoss()
  (item_emb): Embedding(87, 50, padding_idx=86)
  (pos_emb): Embedding(50, 50)
  (emb_dropout): Dropout(p=0.2, inplace=False)
  (attention_layernorms): ModuleList(
    (0): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
    (1): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=50, out_features=50, bias=True)
    )
    (1): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=50, out_features=50, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
    (1): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0): PointWiseFeedForward(
      (conv1): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.2, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(50, 

## Model Training

In [6]:
trainer = trainer_factory(args, model, train_loader, val_loader, test_loader, ckpt_root, dataset.data)
trainer.train()

/home/zhankui/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhankui/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhankui/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhankui/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

Update Best NDCG@10 Model


Epoch 20, loss 0.24461 : 100%|██████████| 16/16 [00:00<00:00, 28.62it/s]
VAL N@5 0.94493: , N@10 0.94671: , N@20 0.94891: , R@5 0.97461: , R@10 0.98047: , R@20 0.98926: , M 0.93631: , AUC 0.99182: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 144.87it/s]
TEST N@5 0.79299: , N@10 0.79483: , N@20 0.79597: , R@5 0.82945: , R@10 0.83537: , R@20 0.84032: , M 0.78392: , AUC 0.88682: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 144.65it/s]
Epoch 21, loss 0.23461 :   0%|          | 0/16 [00:00<?, ?it/s]

Update Best NDCG@10 Model


Epoch 30, loss 0.17356 : 100%|██████████| 16/16 [00:00<00:00, 24.83it/s]
VAL N@5 0.94405: , N@10 0.94662: , N@20 0.94927: , R@5 0.97070: , R@10 0.97852: , R@20 0.98926: , M 0.93698: , AUC 0.99163: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 148.45it/s]
TEST N@5 0.79827: , N@10 0.79960: , N@20 0.80191: , R@5 0.83550: , R@10 0.83940: , R@20 0.84917: , M 0.78873: , AUC 0.88113: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 150.02it/s]
Epoch 40, loss 0.18526 : 100%|██████████| 16/16 [00:00<00:00, 28.33it/s]
VAL N@5 0.94074: , N@10 0.94319: , N@20 0.94492: , R@5 0.97058: , R@10 0.97845: , R@20 0.98529: , M 0.93235: , AUC 0.99098: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 133.88it/s]
TEST N@5 0.80164: , N@10 0.80261: , N@20 0.80381: , R@5 0.84063: , R@10 0.84356: , R@20 0.84844: , M 0.79106: , AUC 0.87683: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 120.26it/s]
Epoch 50, loss 0.16089 : 100%|██████████| 16/16 [00:00<00:00, 26.64it/s]
VAL N@5 0.93433: , N@10

Epoch 100, loss 0.11943 : 100%|██████████| 16/16 [00:00<00:00, 24.14it/s]
VAL N@5 0.92947: , N@10 0.93195: , N@20 0.93590: , R@5 0.96472: , R@10 0.97253: , R@20 0.98822: , M 0.91966: , AUC 0.98911: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 148.62it/s]
TEST N@5 0.79006: , N@10 0.79346: , N@20 0.79557: , R@5 0.83471: , R@10 0.84545: , R@20 0.85424: , M 0.77825: , AUC 0.87558: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 140.72it/s]


## Model Test and Saving

In [7]:
# model testing and saving
trainer.test()
trainer.logger_service.complete({'state_dict': (trainer._create_state_dict())})
trainer.writer.close()

FINAL TEST: N@5 0.79260, N@10 0.79445, N@20 0.79535, R@5 0.83049, R@10 0.83641, R@20 0.84032, M 0.78300, AUC 0.88661, loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 144.50it/s]

Test best model with test set!
{'NDCG@5': 0.7926026657223701, 'Recall@5': 0.8304876498878002, 'NDCG@10': 0.7944467924535275, 'Recall@10': 0.8364100307226181, 'NDCG@20': 0.7953473031520844, 'Recall@20': 0.8403162807226181, 'MRR': 0.7830023355782032, 'AUC': 0.886611957103014}


## Model Serving
We deploy a model with [faiss](https://github.com/facebookresearch/faiss) to accelerate the maximal inner product search. 

In [14]:
import os
import faiss

model = model_factory(args)
model = load_pretrained_weights(model, os.path.join(ckpt_root, 'models', 'best_acc_model.pth'))
model = model.eval()

In [15]:
import numpy as np
d = args.trm_hidden_dim                                 # dimension
nb = model.item_emb.weight.size(0)                      # item pool size
xb = model.item_emb.weight.data.cpu().numpy()           # item embeddings table

In [16]:
index = faiss.index_factory(d, 'HNSW32', faiss.METRIC_INNER_PRODUCT)   # build the index
index.add(xb)                                                          # add vectors to the index

### Model Input Example

In [41]:
x = torch.LongTensor([dataset.train[0]])                # input example
l = torch.LongTensor([len(dataset.train[0])-1])         # length

In [23]:
xq = model(x, length=l, mode='serving').detach().cpu()

### Exact Search via Naive Matmul

In [34]:
k = 10

V, I = torch.topk((torch.Tensor(xb) @ xq.squeeze()), k)
print(I)

tensor([63, 29, 28, 61, 36,  6, 49, 62, 12, 39])


### Approximate Search Via Faiss

In [40]:
V, I = index.search(xq.numpy(), k)
print(I)

[[63 29 28 36 61  6 62 12 49 39]]
